# Part 1 (Problem):

Where in Mumbai would it be best to open a restaurant. What is the place that has the most reviews, what location seems to be the most popular. Is there a type of restaurant that is more popular than another?

The problem will be solved using the Foursquare API, looking at the usercount and tipcount. This problem will help everyone that wants to open up a restaurant in Mumbai and wants to know if it is a viable option where they intend on opening.

# Part 2 (Data):

The data that will be used is Foursquare data for Rio de Janeiro city. It will focus on venues of restaurant type. The popularity will be an indicator for how good an area is for a restaurant.

In [2]:
!pip install geocoder

     |████████████████████████████████| 102kB 6.9MB/s ta 0:00:011


In [3]:
!conda install -c conda-forge folium=0.5.0 --yes

print('Libraries installed!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [5]:
address = 'Mumbai'

geolocator = Nominatim(user_agent="mummbai_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai is 19.0759899, 72.8773928.


In [6]:
CLIENT_ID = 'IFEDZCK152B0LLQEH53TVZTBTJHK5BIGWTXBTV1HXXTBNEYI' # your Foursquare ID
CLIENT_SECRET = 'EAOTQGZGA0H5ZL5CGO2WTPZ50QM10WCVDEFXC5JOVVBJ2QIX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IFEDZCK152B0LLQEH53TVZTBTJHK5BIGWTXBTV1HXXTBNEYI
CLIENT_SECRET:EAOTQGZGA0H5ZL5CGO2WTPZ50QM10WCVDEFXC5JOVVBJ2QIX


In [7]:
search_query = 'Restaurant'
radius = 50000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [9]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=IFEDZCK152B0LLQEH53TVZTBTJHK5BIGWTXBTV1HXXTBNEYI&client_secret=EAOTQGZGA0H5ZL5CGO2WTPZ50QM10WCVDEFXC5JOVVBJ2QIX&ll=19.0759899,72.8773928&v=20180605&query=Restaurant&radius=50000&limit=100'

In [10]:
results = requests.get(url).json()

In [11]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",False,4d4be7dce1ec6dcb0874d275,NaN,IN,Mumbai,India,NaN,820,"[Mumbai, Mahārāshtra, India]","[{'label': 'display', 'lat': 19.07463444538323...",19.074634,72.869724,NaN,Mahārāshtra,Mahesh Restaurant & Bar,v-1595760139,NaN
1,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",False,4ffff479e4b0da072a4c510e,"Ostwal orbit,shop no 2\3,Kanakia junction",IN,Mumbai,India,Mira bhayandar road,27,"[Ostwal orbit,shop no 2\3,Kanakia junction (Mi...","[{'label': 'display', 'lat': 19.0759837, 'lng'...",19.075984,72.877656,NaN,Mahārāshtra,Subway restaurant,v-1595760139,NaN
2,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,511776a6e4b0c785a054bc7b,NaN,IN,NaN,India,NaN,90,[India],"[{'label': 'display', 'lat': 19.07670593261718...",19.076706,72.877808,NaN,NaN,jhakas Family Restaurant,v-1595760139,NaN
3,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,4dc6a998b0fb5556cd0e99a2,Above Shoppers' Stop,IN,Mumbai,India,"Near Amar Mahal, Chembur- Ghatkopar",2534,"[Above Shoppers' Stop (Near Amar Mahal, Chembu...","[{'label': 'display', 'lat': 19.06966328826220...",19.069663,72.900535,400089,Mahārāshtra,IVY Restaurant & Banquets,v-1595760139,72795032
4,"[{'id': '54135bf5e4b08f3d2429dfe0', 'name': 'M...",False,5960987f5c6838397392b821,Ambika Darshan nxt to Neelkanth Corporate park...,IN,Mumbai,India,kirol premier road,1887,[Ambika Darshan nxt to Neelkanth Corporate par...,"[{'label': 'display', 'lat': 19.081892, 'lng':...",19.081892,72.894210,400086,Mahārāshtra,KYROS A Multi Cuisine Restaurant,v-1595760139,NaN


In [12]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Mahesh Restaurant & Bar,Bar,NaN,IN,Mumbai,India,NaN,820,"[Mumbai, Mahārāshtra, India]","[{'label': 'display', 'lat': 19.07463444538323...",19.074634,72.869724,NaN,Mahārāshtra,4d4be7dce1ec6dcb0874d275
1,Subway restaurant,Sandwich Place,"Ostwal orbit,shop no 2\3,Kanakia junction",IN,Mumbai,India,Mira bhayandar road,27,"[Ostwal orbit,shop no 2\3,Kanakia junction (Mi...","[{'label': 'display', 'lat': 19.0759837, 'lng'...",19.075984,72.877656,NaN,Mahārāshtra,4ffff479e4b0da072a4c510e
2,jhakas Family Restaurant,Indian Restaurant,NaN,IN,NaN,India,NaN,90,[India],"[{'label': 'display', 'lat': 19.07670593261718...",19.076706,72.877808,NaN,NaN,511776a6e4b0c785a054bc7b
3,IVY Restaurant & Banquets,Restaurant,Above Shoppers' Stop,IN,Mumbai,India,"Near Amar Mahal, Chembur- Ghatkopar",2534,"[Above Shoppers' Stop (Near Amar Mahal, Chembu...","[{'label': 'display', 'lat': 19.06966328826220...",19.069663,72.900535,400089,Mahārāshtra,4dc6a998b0fb5556cd0e99a2
4,KYROS A Multi Cuisine Restaurant,Multicuisine Indian Restaurant,Ambika Darshan nxt to Neelkanth Corporate park...,IN,Mumbai,India,kirol premier road,1887,[Ambika Darshan nxt to Neelkanth Corporate par...,"[{'label': 'display', 'lat': 19.081892, 'lng':...",19.081892,72.894210,400086,Mahārāshtra,5960987f5c6838397392b821


In [13]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

It is better to open a restaurant near Kurla as it seems to be a happening place.